<a href="https://colab.research.google.com/github/HowlLion/EtherwalletFinder/blob/main/EtherwalletFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydantic-settings asyncio aiohttp nest_asyncio rich tenacity prometheus_client cryptography tabulate

In [1]:
import asyncio
import aiohttp
import time
from tabulate import tabulate

# Set your Etherscan API key here
ETHERSCAN_API_KEY = "WSR7AKCAD7HR8IYTV2GZXQBYPZU7VKPY2F"
BATCH_SIZE = 5  # Reduce batch size if you face rate limiting issues
MAX_SCANS = 20  # Set maximum number of wallet scans for testing purposes


async def check_ETH_balance(session, addresses, etherscan_api_key):
    """Fetch ETH balances in batches."""
    balances = {}
    print("\n🔍 Fetching balances for scanned addresses...")
    start_time = time.time()  # Start measuring time
    for i in range(0, len(addresses), BATCH_SIZE):
        batch = addresses[i:i + BATCH_SIZE]
        addresses_str = ",".join(batch)
        url = f"https://api.etherscan.io/api?module=account&action=balancemulti&address={addresses_str}&tag=latest&apikey={etherscan_api_key}"

        async with session.get(url) as response:
            if response.status == 200:
                try:
                    data = await response.json()
                    if data.get("status") == "1":
                        for acc in data.get("result", []):
                            balances[acc["account"]] = int(acc["balance"]) / 1e18
                    else:
                        print(f"⚠️ API returned error: {data}")
                except Exception as e:
                    print(f"❌ Error decoding JSON: {e}")
            else:
                print(f"❌ HTTP Error: {response.status}")

        await asyncio.sleep(1)  # Prevent API rate limiting
    print(f"✅ Time taken for balance fetching: {time.time() - start_time:.2f} seconds")  # Log the time taken
    return balances


async def get_recent_transactions(session, etherscan_api_key, address):
    """Fetch new addresses from transactions."""
    start_time = time.time()  # Start measuring time
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&sort=desc&apikey={etherscan_api_key}"

    async with session.get(url) as response:
        try:
            data = await response.json()
            if data.get("status") == "1":
                transactions = data.get("result", [])
                print(f"Scanned {len(transactions)} transactions for {address}.")  # Log the number of transactions
                return {tx["from"] for tx in transactions} | {tx["to"] for tx in transactions}
        except Exception as e:
            print(f"❌ Error fetching transactions: {e}")
    print(f"✅ Time taken for transaction scanning: {time.time() - start_time:.2f} seconds")  # Log the time taken
    return set()


async def scan_wallets():
    """Main function to scan wallets, retrieve balances, and print results."""
    valid_addresses = ["0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0", "0x281055afc982d96fab65b3a49cac8b878184cb16", "0x742d35Cc6634C0532925a3b844Bc454e4438f44e"]

    async with aiohttp.ClientSession() as session:
        # Fetch balances for scanned addresses
        balances = await check_ETH_balance(session, valid_addresses, ETHERSCAN_API_KEY)

        # Prepare data for the table
        table_data = []
        for address, balance in balances.items():
            table_data.append([address, f"{balance:.4f} ETH"])

        # Print beautified balance table using tabulate
        print("\n📊 Wallet Balances:")
        print(tabulate(table_data, headers=["Address", "Balance (ETH)"], tablefmt="fancy_grid"))

        # Now fetch new addresses from transactions
        all_scanned_addresses = set(valid_addresses)
        for address in valid_addresses:
            transactions = await get_recent_transactions(session, ETHERSCAN_API_KEY, address)
            all_scanned_addresses.update(transactions)

        # Display the total number of addresses scanned
        print(f"\n🔎 Total addresses scanned: {len(all_scanned_addresses)}")


# Handle event loop for Jupyter Notebook
try:
    await scan_wallets()
except RuntimeError:
    asyncio.run(scan_wallets())



🔍 Fetching balances for scanned addresses...
✅ Time taken for balance fetching: 1.14 seconds

📊 Wallet Balances:
╒════════════════════════════════════════════╤═════════════════╕
│ Address                                    │ Balance (ETH)   │
╞════════════════════════════════════════════╪═════════════════╡
│ 0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0 │ 55381.4252 ETH  │
├────────────────────────────────────────────┼─────────────────┤
│ 0x281055afc982d96fab65b3a49cac8b878184cb16 │ 0.0189 ETH      │
├────────────────────────────────────────────┼─────────────────┤
│ 0x742d35Cc6634C0532925a3b844Bc454e4438f44e │ 459496.1862 ETH │
╘════════════════════════════════════════════╧═════════════════╛
Scanned 10000 transactions for 0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0.
Scanned 523 transactions for 0x281055afc982d96fab65b3a49cac8b878184cb16.
Scanned 10000 transactions for 0x742d35Cc6634C0532925a3b844Bc454e4438f44e.

🔎 Total addresses scanned: 7418
